<a href="https://colab.research.google.com/github/Alirez1043/DeepLearning/blob/main/Assighments/4/catVSdog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir =  "/root/.keras/datasets/cats_and_dogs_filtered/train"
validation_dir = "/root/.keras/datasets/cats_and_dogs_filtered/validation"

68616192/68606236 [==============================] - 1s 0us/step


In [2]:
train_ds = tf.data.Dataset.list_files(str(train_dir+'/*/*'))
val_ds = tf.data.Dataset.list_files(str(validation_dir+'/*/*'))

In [3]:
print("Examples of Train Path :")
for path in train_ds.take(2):
  print(path)

print("\nExamples of Validation path :")
for path in val_ds.take(2) :
  print(path)

Examples of Train Path :
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/cats/cat.550.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/train/dogs/dog.592.jpg', shape=(), dtype=string)

Examples of Validation path :
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/validation/cats/cat.2176.jpg', shape=(), dtype=string)
tf.Tensor(b'/root/.keras/datasets/cats_and_dogs_filtered/validation/dogs/dog.2112.jpg', shape=(), dtype=string)


In [4]:
def prepare_labeled_data(path):
  class_names = tf.constant(['cat', 'dog'])

  categorical_label = tf.strings.split(path, '/')[-1]
  categorical_label = tf.strings.split(categorical_label, '.')[0]
  label = tf.cast(categorical_label == class_names[0], tf.int8) # cat=1, dog=0
  image = tf.io.read_file(path)
  image = tf.image.decode_image( image, expand_animations=False )
  
  return image, label

In [5]:
def normalization(img):
  img = tf.image.convert_image_dtype(img, tf.float32)
  return img / 255

In [6]:
def augment_data(image):
    image = tf.image.resize_with_crop_or_pad(image, 180, 180) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[150, 150, 3]) # Random crop back to 28x28
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
    image = tf.image.flip_left_right(image)
    return image

In [7]:
def load_image_train(path) :
    image  ,label = prepare_labeled_data(path)
    image = augment_data(image)
    image = normalization(image)
    return image  ,label

def load_image_test(path) :
    image, label = prepare_labeled_data(path)
    image = tf.image.resize(image, [150, 150])
    image = normalization(image)
    return image, label

In [8]:
BATCH_SIZE=64
AUTOTUNE = tf.data.AUTOTUNE
BUFFER_SIZE=1000

In [9]:
train_ds = train_ds.map(load_image_train).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_ds = val_ds.map(load_image_test).batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [ ]:
print("Instances of train images :")
for img, lbl in train_ds.take(1):
  print(img[0] ,"   " ,lbl[0])
print("\ninstances of validation images :")
for img, lbl in val_ds.take(1):
  print(img[0] ,"   " ,lbl[0])

In [19]:
class Conv2DBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters ,activation = 'relu'  ,padding = 'valid',**kwargs) :
    super(Conv2DBlock ,self).__init__(**kwargs)
    self.conv2d = tf.keras.layers.Conv2D(num_filters ,(3,3),padding = padding ,activation = activation)
    self.maxPool2D = tf.keras.layers.MaxPool2D((2,2))

  def call(self ,input) :
    Z = input
    Z = self.conv2d(Z)
    Z = self.maxPool2D(Z)
    return Z 

In [20]:
class MyCustomModel(tf.keras.models.Model):
  def __init__(self ,activation = 'relu' ,**kwargs):
    super(MyCustomModel ,self).__init__(**kwargs)
    self.blocks = [
                   Conv2DBlock(32),
                   Conv2DBlock(64),
                   Conv2DBlock(128),
                   Conv2DBlock(128)
                  ]
    self.model_layers = [
                   tf.keras.layers.Flatten(),
                   tf.keras.layers.Dropout(0.2),
                   tf.keras.layers.Dense(512, activation=activation),
                   tf.keras.layers.Dense(128 ,activation = activation),
                   tf.keras.layers.Dense(1),
                   tf.keras.layers.Activation('sigmoid')
                  ]
  def call(self ,input) :
    Z = input
    for block in self.blocks :
      Z = block(Z)
    for layer in self.model_layers :
      Z = layer(Z)
    return Z 

In [21]:
my_model = MyCustomModel()

In [24]:
my_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [25]:
history = my_model.fit(train_ds, 
                    epochs=25,
                    validation_data=val_ds)

Epoch 1/25
32/32 [==============================] - 12s 227ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6974 - val_accuracy: 0.5000
Epoch 2/25
32/32 [==============================] - 8s 191ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6975 - val_accuracy: 0.5000
Epoch 3/25
32/32 [==============================] - 8s 193ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6974 - val_accuracy: 0.5000
Epoch 4/25
32/32 [==============================] - 9s 193ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6975 - val_accuracy: 0.5000
Epoch 5/25
32/32 [==============================] - 8s 192ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6975 - val_accuracy: 0.5000
Epoch 6/25
32/32 [==============================] - 8s 193ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6975 - val_accuracy: 0.5000
Epoch 7/25
32/32 [==============================] - 8s 195ms/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6976 - val_accuracy: 0.5000
Epoch